<h2> Embedding

In [ ]:
from torch import nn
import torch
import math
import pandas as pd
from torch.nn import functional as F
import copy
from torch.utils.data import Dataset,DataLoader
import time
from tqdm.notebook import tqdm
import numpy as np
import time

In [ ]:
class Embedder(nn.Module): #임베딩 레이어
    def __init__(self,vocab_size,d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,d_model)
    
    def forward(self,x):
        #x= x.to("cpu")
        return self.embed(x)

In [ ]:
e = Embedder(32000,768)

In [ ]:
idx = torch.tensor([1,2,3])
e(idx)

<h2> Positional Encoding

In [ ]:
import math
import torch

In [ ]:
class PositionalEncoder(nn.Module): #위치 인코딩 레이어
    def __init__(self, d_model, max_seq_len = 128):
        super().__init__()
        self.d_model = d_model
        
        #위치 인코딩 초기화
        positional_encoding = torch.zeros(max_seq_len, d_model)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                positional_encoding[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        positional_encoding = positional_encoding.unsqueeze(0)
        #잔차 연결
        self.register_buffer('positional_encoding', positional_encoding)
 
    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(1)
        x = x + self.positional_encoding[:,:seq_len]
        return x

In [ ]:
p = PositionalEncoder(768)

In [ ]:
#y_pred값 대충 생성
input_seq = torch.randint(0,40,(128,))
emb = e(input_seq)
emb.shape

In [ ]:
y_pred = p(emb.to("cpu"))

In [ ]:
#y값 대충 생성
input_seq2 = torch.randint(0,40,(128,))
emb2 = e(input_seq2)

In [ ]:
y = p(emb2)

In [ ]:
loss = torch.mean((y_pred - y)**2)
loss.backward()

<h2> Masking

In [ ]:
def get_attn_mask(input_seq,input_pad):
    return (input_seq != input_pad).unsqueeze(1).to("cuda")

In [ ]:
def get_target_attn_mask(target_seq,target_pad):
    target_mask = (target_seq != target_pad).unsqueeze(1)
    size = target_seq.size(1)
    nopeak_mask = np.triu(np.ones((1,size,size)),k=1).astype('uint8')
    nopeak_mask = torch.from_numpy(nopeak_mask) == 0
    nopeak_mask = nopeak_mask.to("cuda")
    target_mask = target_mask & nopeak_mask
    return target_mask

<h2> Self-Attention

In [ ]:
from torch.nn import functional as F

In [ ]:
def attention(query, key, value, d_k, attention_mask=False, dropout=None):
    
    scores = torch.matmul(query,key.transpose(-2,-1)) /  math.sqrt(d_k)
    
    if attention_mask is True:
        attention_mask = get_attn_mask(scores,0)
        attention_mask = attention_mask.unsqueeze(1)
        scores = scores.masked_fill(attention_mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, value)
    return output

<h2> Multi-Headed Attention

In [ ]:
from scipy.special import softmax

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,heads,d_model,dropout_rate = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.query_layer = nn.Linear(d_model,d_model)
        self.key_layer = nn.Linear(d_model,d_model)
        self.value_layer = nn.Linear(d_model,d_model)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.out_layer = nn.Linear(d_model,d_model)
        
    def forward(self,query,key,value,attention_mask = True):
        
        batch_size = query.size(0)
        
        query = self.key_layer(query).view(batch_size,-1,self.h,self.d_k)
        key = self.key_layer(key).view(batch_size,-1,self.h,self.d_k)
        value = self.key_layer(value).view(batch_size,-1,self.h,self.d_k)
        
        query =  query.transpose(1,2)
        key = key.transpose(1,2)
        value = value.transpose(1,2)
        
        scores = attention(query,key,value,self.d_k,attention_mask,self.dropout)
        Z = scores.transpose(1,2).contiguous().view(batch_size,-1,self.d_model)
        output = self.dropout(Z)
        
        return output

In [ ]:
mha = MultiHeadAttention(8,512)

In [ ]:
query = torch.rand((1,3,512))
key = torch.rand((1,3,512))
value = torch.rand((1,3,512))

<h2> Feed-Forward

In [ ]:
class FeedForward(nn.Module):
    def __init__(self,d_model,d_ff = 2048,dropout = 0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(d_model,d_ff)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(d_ff,d_model)
        
    def forward(self,x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

<h2> Normalisation

<ul>
    <li>prevents the range of value in the layers changing too much
<li>it makes model trains faster and has better ability</ul>

In [ ]:
class Norm(nn.Module):
    def __init__(self,d_model,eps = 1e-6):
        super().__init__()
        
        self.size = d_model
        
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    
    def forward(self,x):
        norm = self.alpha *(x - x.mean(dim = -1,keepdim=True)) / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

<h1> ★Encoder&Decoder Layer★

<ui>
<li> build an encoder layer with one multi-head attention layer and one feed-forward layer </li>
</ui>

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,d_model,heads,dropout = 0.1):
        super().__init__()
        self.norm1 = Norm(d_model)
        self.norm2 = Norm(d_model)
        self.mha = MultiHeadAttention(heads,d_model)
        self.ff = FeedForward(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self,x,mask):
        x2 = self.norm1(x)
        x = x + self.dropout1(self.mha(x2,x2,x2,mask))
        x2 = self.norm2(x)
        x = x + self.dropout2(self.ff(x2))
        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,d_model,heads,dropout=0.1):
        super().__init__()
        self.norm1 = Norm(d_model)
        self.norm2 = Norm(d_model)
        self.norm3 = Norm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
        self.mha1 = MultiHeadAttention(heads, d_model)
        self.mha2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        
    def forward(self,x,encoder_outputs,src_mask,trg_mask):
        x2 = self.norm1(x)
        x = x + self.dropout1(self.mha1(x2,x2,x2,trg_mask))
        
        x2 = self.norm2(x)
        x = x + self.dropout2(self.mha2(x2,encoder_outputs,encoder_outputs,src_mask))
        
        x2 = self.norm3(x)
        x = x + self.dropout3(self.ff(x2))
        return x
        

In [ ]:
# build a conveinient cloning function that can generate multiple layers
import copy
def get_clones(module,N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

<h1> Encoder Decoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,d_model,N,heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size,d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model,heads),N)
        self.norm = Norm(d_model)
    
    def forward(self,src,mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x,mask)
        return self.norm(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,d_model,N,heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size,d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model,heads),N)
        self.norm = Norm(d_model)
    def forward(self,target,encoder_outputs,src_mask,target_mask):
        x = self.embed(target)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x,encoder_outputs,src_mask,target_mask)
        return self.norm(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [ ]:
from tokenizers import Tokenizer
from transformers import FlaubertTokenizer

eng_tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
french_tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

eng_vocab = eng_tokenizer.get_vocab()
french_vocab = french_tokenizer.get_vocab()

In [ ]:
d_model = 512
heads = 8
N = 6
batch_size = 24
epochs = 50

src_vocab = len(eng_vocab)
trg_vocab = len(french_vocab)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
model = model.to("cuda")

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

<h2> Dataset

In [ ]:
class Dataset(Dataset):
    def __init__(self, dataset, eng_tokenizer,french_tokenizer,max_len = 64,source = "source_language",target = "target_language"):
        self.max_len = max_len
        self.sources = []
        self.targets = []
        self.source_lengths = []
        
        for i,row in dataset.iterrows():
            ids = eng_tokenizer.encode(row[source]).ids
            self.sources.append(self.padding(ids))
            self.source_lengths.append(len(ids))
            self.targets.append(self.padding(french_tokenizer.encode(row[target])))
        
    def padding(self,ids):
        pad = np.zeros(self.max_len, dtype=np.int64)
        pad[:len(ids)] = ids
        return pad
    
    def __getitem__(self, i):
        return {"source" : self.sources[i],
                "target" : self.targets[i],
                "source_length" : self.source_lengths[i]
               }

    def __len__(self):
        return (len(self.sources))

In [ ]:
df = pd.read_csv("simplest_eng_fra.csv")

In [ ]:
train_df = df[df.split=='train']
train_size = len(train_df)
val_df = df[df.split=='val']
validation_size = len(val_df)
test_df = df[df.split=='test']
test_size = len(test_df)

In [ ]:
train_dataset = Dataset(train_df,eng_tokenizer,french_tokenizer)
valid_dataset = Dataset(val_df,eng_tokenizer,french_tokenizer)

<h2> DataLoader

In [ ]:
from torch.utils.data import DataLoader
def load_data(dataset, batch_size, shuffle=True, 
                            drop_last=True, device="cuda"):

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        lengths = data_dict['source_length'].numpy()
        sorted_length_indices = lengths.argsort().tolist()
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name][sorted_length_indices].to(device)
        yield out_data_dict

In [ ]:
start = time.time()
temp = start
total_loss=0
print_every=100
for epoch in tqdm(range(epochs),total=epochs):
    model.train()
    
    
    batch_generator = load_data(train_dataset, batch_size)
    
    for i, batch in tqdm(enumerate(batch_generator),total=train_size/batch_size):
        optim.zero_grad()
        
        src = batch['source']
        trg = batch['target']
        
        trg_input = trg[:, :-1]
        targets = trg[:, 1:].contiguous().view(-1)

        src_mask = get_attn_mask(src,input_pad = 0)
        trg_mask = get_target_attn_mask(trg_input,target_pad = 0)
        preds = model(src, trg_input, src_mask, trg_mask)
        preds = preds.view(-1, preds.size(-1)).to("cuda")
        
        result = trg[:, 1:].contiguous().view(-1)
        loss = F.cross_entropy(preds,result, ignore_index=0)
        
        loss.backward()
        optim.step()

        total_loss += loss.item()
        
        if (i + 1) % print_every == 0:
            loss_avg = total_loss / print_every
            print("time = %dm, epoch %d, iter = %d, loss = %.3f,%ds per %d iters" % \
                  ((time.time() - start) // 60,epoch + 1, i + 1, loss_avg, time.time() - temp,print_every))
            total_loss = 0
            temp = time.time()
            
    batch_generator = load_data(valid_dataset, batch_size)
    running_loss = 0.
    model.eval()

    for i, batch in tqdm(enumerate(batch_generator),total=validation_size/batch_size):
        
        src = batch['source']
        trg = batch['target']
        
        trg_input = trg[:, :-1]
        targets = trg[:, 1:].contiguous().view(-1)
        
        src_mask = get_attn_mask(src,input_pad = 0)
        trg_mask = get_target_attn_mask(trg_input,target_pad = 0)
        
        preds = model(src, trg_input, src_mask, trg_mask)
        preds = preds.view(-1, preds.size(-1)).to("cuda")
        result = trg[:, 1:].contiguous().view(-1)
        loss = F.cross_entropy(preds,result, ignore_index=0)
        running_loss += (loss.item() - running_loss) / (i + 1)
        
    print("validation: epoch %d, loss = %.3f," % (epoch + 1, running_loss) )

In [78]:
def get_source_sentence(source_vocab, batch_dict, index):
    indices = batch_dict['source'][index].cpu().data.numpy()
    return sentence_from_indices(indices, source_vocab)

def get_true_sentence(target_vocab, batch_dict, index):
    return sentence_from_indices(batch_dict['target'].cpu().data.numpy()[index], target_vocab)
    
def get_sampled_sentence(target_vocab, batch_dict, index):
    y_pred = model(batch_dict['source'], batch_dict['target'],False,False )
    return sentence_from_indices(torch.max(y_pred, dim=2)[1].cpu().data.numpy()[index], target_vocab)

def get_all_sentences(source_vocab,target_vocab, batch_dict, index):
    return {"source": get_source_sentence(source_vocab, batch_dict, index), 
            "truth": get_true_sentence(target_vocab, batch_dict, index), 
            "sampled": get_sampled_sentence(target_vocab, batch_dict, index)}
    
def sentence_from_indices(indices, vocab, strict=True):
    ignore_indices = set([0,1,2,5])
    vocab_map = dict(map(reversed,vocab.items()))
    out = []
    for index in indices:
        if index == 0 and strict:
            continue
        elif index == 1 and strict:
            return " ".join(out)
        else:
            out.append(vocab_map[index])
    return " ".join(out)

In [41]:
test_dataset = Dataset(test_df,eng_tokenizer,french_tokenizer)
batch_generator = load_data(test_dataset, 
                                       batch_size=batch_size)
batch_dict = next(batch_generator)

model = model.eval().to("cuda")

In [42]:
source = "There is no need It is my treat"
target = "Il n'y a pas besoin C'est mon régal"

In [65]:
source_vector = [0 for i in range(64)]
source_ids = eng_tokenizer.encode(source).ids
source_vector[:len(source_ids)] = source_ids
source_vector = np.array(source_vector)

In [66]:
target_vector = [0 for i in range(64)]
target_ids = french_tokenizer.encode(target)
target_vector[:len(target_ids)] = target_ids
target_vector = np.array(target_vector)

In [67]:
vector = {'source_vector' : target_vector,
         'target_vector' : source_vector}

In [68]:
batch_dict['source'][1] = torch.from_numpy(vector['source_vector'])
batch_dict['target'][1] = torch.from_numpy(vector['target_vector'])

In [79]:
results = get_all_sentences(eng_vocab,french_vocab, batch_dict, 1)

In [83]:
for i in range(24):
    print(get_all_sentences(eng_vocab,french_vocab, batch_dict, i))

{'source': '[CLS] he is foolish . [SEP]', 'truth': 'il</w> est</w> idiot</w> .</w>', 'sampled': 'il</w> est</w> idiot</w> .</w>'}
{'source': '[unused58] [unused50] [unused65] [unused33] [unused41] [unused401] [unused108] [unused26] [unused146] ##horpe', 'truth': 'ils</w> commencer</w> spectacle</w> Dé banques</w> température</w> spectacle</w> appui</w> Tél</w> sans</w>', 'sampled': "il</w> commencer</w> ille</w> our our j'</w> ûment</w> ûment</w> sans</w> sans</w> sans</w> sans</w> si</w> si</w> si</w> d'</w> encore</w> par</w> encore</w> encore</w> encore</w> encore</w> encore</w> encore</w> encore</w> je</w> je</w> je</w> je</w> je</w> je</w> je</w> je</w> je</w> je</w> je</w> c'</w> je</w> je</w> je</w> je</w> c'</w> c'</w> c'</w> je</w> je</w> je</w> j'</w> j'</w> j'</w> j'</w> j'</w> j'</w> j'</w> c'</w> je</w> je</w> je</w> je</w> je</w> j'</w> j'</w> j'</w> j'</w>"}
{'source': "[CLS] we ' re coming . [SEP]", 'truth': 'nous</w> venons</w> .</w>', 'sampled': 'nous</w> restons</w> 